# Clasificador binario con vectores densos
Usamos vectores densos de palabra y documento para el problema del análisis de sentimientos en Twitter

## Carga y preparación de los datos

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

# Leemos los datos
df = pd.read_csv('tweets_max.csv', index_col=None)


In [3]:
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

## Limpieza de texto
Hacemos un pequeño pre-procesado del texto antes de extraer las características:  
- Quitamos las menciones y las URL del texto porque no aportan valor para el análisis de sentimientos.
- Los hashtag sí que pueden aportar valor así que simplemente quitamos el #.
- Quitamos los signos de puntuación y palabras menores de 3 caracteres.
- Por último quitamos todos los símbolos de puntuación del texto (que forman parte de un token).
- Lematizamos el texto y lo guardamos en otra columna para comparar resultados del clasificador. 

In [4]:
import re, string, spacy
nlp=spacy.load('es_core_news_md')

In [5]:
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['unos', 'unas', 'algún', 'alguna', 'algunos', 'algunas', 'ese', 'eso', 'así']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens if not (tok in stop_words) and len(tok)>2]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text
    

In [6]:
from tqdm import tqdm
tqdm.pandas()

df["limpio"]=df['content'].progress_apply(clean_text)


100%|██████████| 1449/1449 [00:08<00:00, 163.54it/s]


In [7]:
df["lemas"] = df['content'].progress_apply(clean_text, lemas=True)

100%|██████████| 1449/1449 [00:07<00:00, 199.21it/s]


## Modelo con word embeddings
Ahora vamos a usar como espacio de características los *word vectors* de las palabras de nuestro corpus.  
Como cada palabra tiene un vector de longitud fija, tenemos que obtener un único vector como promedio de todas las palabras del tweet.  
En spaCy, el vector de cada palabra es el atributo `vector`.  
El atributo `vector` del objeto `Doc` del texto procesado en spaCy contiene el vector promedio de todos los tokens.

Vemos el tamaño del vector del modelo `Spacy`

In [8]:
nlp.vocab.vectors_length

300

Es el tamaño del vector de cada token

In [9]:
doc=nlp(df.content[1])
doc[1].vector.shape

(300,)

Que coincide con el tamaño del vector del documento entero:

In [10]:
doc.vector.shape

(300,)

Este vector corresponde al promedio de los vectores de todos los tokens del documento que tienen un vector definido en `spaCy`

In [11]:
#Spacy ya calcula el promedio de los vectores de un documento en Doc.vector
word_embeddings = np.stack([nlp.make_doc(tweet).vector for tweet in df.limpio])

In [12]:
word_embeddings.shape

(1449, 300)

In [13]:
type(word_embeddings)

numpy.ndarray

Generamos los conjuntos de entrenamiento con word embeddings de cada tweet y volvemos a aplicar los mismos clasificadores de antes.

### Ejercicio
Divide las entradas y salidas del modelo en entrenamiento y test respetando la misma división que hemos empleado hasta ahora.

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(word_embeddings,
                                                    df['polarity'],
                                                    test_size = 0.3,
                                                    random_state = 0)

Aplicamos un clasificador a esta matriz de características. En este caso la matriz conviene valores decimales, por lo que el clasificador `MultinomialNB` se tiene que sustituir por un `GaussianNB` para usar un modelo Naïve Bayes, pero también podemos probar otros modelos más complejos (p. ej. un SVM con un kernel RFB)

In [15]:
type(X_train)

numpy.ndarray

In [16]:
X_train.shape

(1014, 300)

In [17]:
X_test.shape

(435, 300)

In [18]:
#entrenamos clasificadores con modelos word embeddings
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

modelos = [('Logistic Regression', LogisticRegression(solver='liblinear')),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5)),
           ('RFB SVM', SVC(gamma='scale', C=2))]

for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')


Modelo Logistic Regression: 0.71
Modelo Naive Bayes: 0.69
Modelo Linear SVM: 0.67
Modelo RFB SVM: 0.77


Los modelos con word embedding promediado para todo el tweet funcionan un poco peor que con vectorizaciones sparse (BoW, TF-IDF) y modelos simples, pero mejoran con modelos ML avanzados. Para usar word embeddings conviene irse a un modelo de aprendizaje profundo (por ejemplo un modelo CNN o un modelo secuencial con LSTM), para lo que es necesario entrenar con un conjunto de datos mucho mayor.  
Una ventaja de los modelos basados en *embeddings* en que generalizan mejor con menos muestran de entrenamiento.

In [19]:
#seleccionamos un subset de los datos de entrenamiento sólo

muestras = 300
rng = np.random.default_rng(0)
idx = rng.choice(X_train.shape[0], muestras, replace=False)

for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train[idx,:], y_train.iloc[idx])
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

Modelo Logistic Regression: 0.72
Modelo Naive Bayes: 0.64
Modelo Linear SVM: 0.71
Modelo RFB SVM: 0.75


Con sólo 300 ejemplos de train es capaz de entregar casi igual de bien, y mejor que casi todos los modelos basados en BoW/TFIDF

### Modelo con texto lematizado
Repetimos con el texto lematizado y los mismo WE de `spaCy`  

#### Ejercicio
- Calcula la matriz de embeddings sobre los textos lematizados.  
- Divide en entrenamiento y test.  
- Entrena los mismos modelos de antes y compara los valores de *accuracy*

In [20]:
word_embeddings = np.stack([nlp.make_doc(tweet).vector for tweet in df['lemas']])

X_train, X_test, y_train, y_test = train_test_split(word_embeddings,
                                                    df['polarity'],
                                                    test_size = 0.3,
                                                    random_state = 0)

for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train[idx,:], y_train.iloc[idx])
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

Modelo Logistic Regression: 0.68
Modelo Naive Bayes: 0.58
Modelo Linear SVM: 0.67
Modelo RFB SVM: 0.72


### Modelo WE de Fasttext
Usamos los WE de Fasttext con `gensim` para ver si mejora la predicción

In [21]:
#carga de vectores en formato TXT
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = '~/Downloads/fasttext-sbwc.100k.vec'
wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec)

FileNotFoundError: [Errno 2] No such file or directory: '/home/vic_263/Downloads/fasttext-sbwc.100k.vec'

In [ ]:
from numpy.linalg import norm

def to_vector(texto):
    """Función para calcular vector semántico
    de un documento"""
    tokens = texto.lower().split()
    vec = np.zeros(300)
    for word in tokens:
        # si la palabra está la acumulamos
        if word in wordvectors:
            vec += wordvectors[word]
    if norm(vec):
        return vec / norm(vec)
    else:
        return vec

Calculamos la matriz de WE sobre el texto sin lematizar con esta función

In [ ]:
word_embeddings = np.stack([to_vector(tweet) for tweet in df.limpio])
word_embeddings.shape

In [ ]:
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(word_embeddings, 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

#entrenamos y validamos
for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

Modelo con WE de FastText sobre texto lematizado

In [ ]:
word_embeddings = np.stack([to_vector(tweet) for tweet in df.lemas])
word_embeddings.shape

In [ ]:
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(word_embeddings, 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

#entrenamos y validamos
for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

### Modelos WE con ponderación IDF
Usamos los valores IDF de cada término para promediar el vector denso del texto a partir de sus WE

In [ ]:
def tfidf_wtd_avg_word_vectors(doc, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF
    a un documento (como string).
    Utiliza los WE de spaCy'''
    tokens = doc.split()

    feature_vector = np.zeros((nlp.vocab.vectors_length,),dtype="float64")
    wts = 0.      
    for word in tokens:
        if nlp.vocab[word].has_vector and word_tfidf_map.get(word, 0): #sólo considera palabras conocidas
            weighted_word_vector = word_tfidf_map[word] * nlp.vocab[word].vector
            wts = wts + 1
            feature_vector = np.add(feature_vector, weighted_word_vector)
    if wts:
        feature_vector = np.divide(feature_vector, wts)
        
    return feature_vector
    
def tfidf_weighted_averaged_word_vectorizer(corpus, word_tfidf_map):
    '''Aplica la función de cálculo del WE ponderado por TF-IDF a todos los
    documentos del corpus (cada doc es una lista de tokens)'''                                       
    features = [tfidf_wtd_avg_word_vectors(doc, word_tfidf_map)
                    for doc in corpus]
    return np.array(features)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

X_train, X_test, y_train, y_test = train_test_split(df.limpio, 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

# características tfidf
tfidf_train_features = tfidf_vectorizer.fit_transform(X_train)  
tfidf_test_features = tfidf_vectorizer.transform(X_test)    

# características tfidf weighted averaged word vector
word_tfidf_map = {key:value for (key, value) in zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)}
tfidf_wv_train_features = tfidf_weighted_averaged_word_vectorizer(corpus=X_train, 
                                                                  word_tfidf_map=word_tfidf_map)

tfidf_wv_test_features = tfidf_weighted_averaged_word_vectorizer(corpus=X_test, 
                                                                 word_tfidf_map=word_tfidf_map)

In [ ]:
tfidf_wv_train_features.shape

In [ ]:
tfidf_wv_test_features.shape

In [ ]:
#entrenamos y validamos
for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(tfidf_wv_train_features, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(tfidf_wv_test_features)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')

### Modelos Sentence Embedding
Probamos con un modelo sentence BERT para obtener los *sentence embeddings* de cada Tweet y entrenar un clasificador ML.

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
sentence_embeddings = model.encode(df['limpio'].to_list()) #por defecto devuelve array numPy
sentence_embeddings.shape

Los embeddings de documento de la libraría `sentence transformer` son de 384 dimensiones

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, 
                                                    df['polarity'], 
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')


Seleccionamos sólo 300 muestras para entrenar y probamos

In [ ]:
for m, clf in modelos:
    #entrenamos sobre train
    clf.fit(X_train[idx,:], y_train.iloc[idx])
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(X_test)
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')


De nuevo con sólo 300 muestras el modelo funciona muy bien. Los modelos basados en vectores semánticos generalizan mucho mejor.  
En cambio con los modelos *sparse* el rendimiento baja más. Partimos del modelo que mejor nos había funcionado (TF-IDF sobre texto lematizado)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizamos
vect = TfidfVectorizer()

X_train, X_test, y_train, y_test = train_test_split(df["lemas"], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

X_train_vectorized = vect.fit_transform(X_train.iloc[idx])
X_test_vectorized = vect.transform(X_test)
X_train_vectorized.shape

In [ ]:
for m, clf in modelos:
    clf.fit(X_train_vectorized.toarray(), y_train.iloc[idx])
    prediccion = clf.predict(X_test_vectorized.toarray())
    print(f'Modelo {m}: {accuracy_score(y_test, prediccion):.2f}')